In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from keras.models import load_model

2024-01-24 23:16:13.724734: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 23:16:13.760471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
imgs_path = "./Train"
data = []
labels = []
CLASSES = 43
# using for loop to access each image
for i in range(CLASSES):
    img_path = os.path.join(imgs_path, str(i)) #0-42
    for img in os.listdir(img_path):
        im = Image.open(imgs_path + '/' + str(i) + '/' + img)
        im = im.resize((32,32))
        im = np.array(im)
        data.append(im)
        labels.append(i)
data = np.array(data)
labels = np.array(labels)
print("data[0]: ",data[0])
print("labels[0: ]",labels[0])

data[0]:  [[[137  97  91]
  [178 113 117]
  [157  67  63]
  ...
  [176  60  41]
  [186 111  66]
  [247 230 225]]

 [[167 114 120]
  [169  89 110]
  [153  48  50]
  ...
  [155  68  46]
  [178 112  57]
  [248 215 188]]

 [[177 114 105]
  [165  75  77]
  [156  45  42]
  ...
  [180 121 102]
  [185 125  95]
  [248 186 183]]

 ...

 [[140 166 198]
  [116 134 166]
  [106 119 101]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[148 191 211]
  [116 149 165]
  [157 183 167]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[100 155 146]
  [ 73 102  86]
  [103 112 100]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
labels[0: ] 0


In [3]:
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
print("training shape: ",x_train.shape, y_train.shape)
print("testing shape: ",x_val.shape, y_val.shape)
# convert interge label to one-hot data
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

print(y_train[1])

training shape:  (31367, 32, 32, 3) (31367,)
testing shape:  (7842, 32, 32, 3) (7842,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
from sklearn.metrics import accuracy_score
test = pd.read_csv("./Test.csv")
test_labels = test['ClassId'].values.tolist()

test_img_path = "./"
test_imgs = test['Path'].values
test_data = []

for img in test_imgs:
    im = Image.open(test_img_path + '/' + img)
    im = im.resize((32,32))
    im = np.array(im)
    test_data.append(im)
test_data = np.array(test_data)

In [4]:
from tensorflow.keras import layers, models

# Transfer Learning using ResNet50

In [5]:
from tensorflow.keras.applications.resnet import ResNet50


resnet = ResNet50(weights= 'imagenet', include_top=False, input_shape= (32,32,3))

In [6]:
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model


x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(43, activation= 'softmax')(x)
model = Model(inputs = resnet.input, outputs = predictions)

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

model_check = ModelCheckpoint('convnet_for_GTSRB_with_ResNet50_ImageNet_weights.keras', monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=0, mode='max', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

csv_logger = CSVLogger('train_log.csv', separator=',')

In [9]:
n_epochs = 50
history =  model.fit(x_train, y_train,  batch_size = 32, epochs = n_epochs, verbose = 1, 
              validation_data = (x_val, y_val), callbacks = [model_check, early, reduce_lr, csv_logger])

Epoch 1/50
981/981 [==============================] - 603s 601ms/step - loss: 3.0987 - accuracy: 0.3310 - val_loss: 2.1936 - val_accuracy: 0.3577 - lr: 0.0010
Epoch 2/50
981/981 [==============================] - 589s 601ms/step - loss: 1.9523 - accuracy: 0.5401 - val_loss: 55.7118 - val_accuracy: 0.2464 - lr: 0.0010
Epoch 3/50
981/981 [==============================] - 590s 602ms/step - loss: 0.7296 - accuracy: 0.7808 - val_loss: 0.2890 - val_accuracy: 0.9045 - lr: 0.0010
Epoch 4/50
981/981 [==============================] - 833s 850ms/step - loss: 0.3619 - accuracy: 0.9024 - val_loss: 0.2652 - val_accuracy: 0.9185 - lr: 0.0010
Epoch 5/50
981/981 [==============================] - 876s 893ms/step - loss: 0.4631 - accuracy: 0.8850 - val_loss: 0.2652 - val_accuracy: 0.9211 - lr: 0.0010
Epoch 6/50
981/981 [==============================] - 871s 888ms/step - loss: 0.2389 - accuracy: 0.9381 - val_loss: 0.8435 - val_accuracy: 0.8279 - lr: 0.0010
Epoch 7/50
981/981 [=========================

In [10]:
model.save('convnet_for_GTSRB_with_ResNet50_ImageNet_weights.keras')
# model.save('convnet_for_GTSRB_with_ResNet50.h5')

In [14]:
from tensorflow import keras
test_model = keras.models.load_model("convnet_for_GTSRB_with_ResNet50_ImageNet_weights.keras")
predictions = test_model.predict(test_data)
classes_x = np.argmax(predictions, axis = 1).tolist()
classes_x = np.array([classes_x]).tolist()[0]

print("Accuracy on test dataset using CNN after transfer learning with ResNet50 using ImageNet weights: ", accuracy_score(test_labels, classes_x))

395/395 [==============================] - 69s 170ms/step
Accuracy on test dataset using CNN after transfer learning with ResNet50 using ImageNet weights:  0.94972288202692
